In [15]:
import arcpy
import requests
import pandas as pd
import os
import io
import zipfile
import datetime
import numpy as np

In [16]:
workspace = r"C:\Users\cason\Desktop\GIS5571\Lab3"

In [17]:
url = r"https://ndawn.ndsu.nodak.edu/table.csv?station=78&station=111&station=98&station=174&station=142&station=138&station=161&station=9&station=10&station=118&station=56&station=11&station=12&station=58&station=13&station=84&station=55&station=7&station=87&station=14&station=15&station=96&station=16&station=137&station=124&station=143&station=17&station=85&station=140&station=134&station=18&station=136&station=65&station=104&station=99&station=19&station=129&station=20&station=101&station=81&station=21&station=97&station=22&station=75&station=2&station=172&station=139&station=23&station=62&station=86&station=24&station=89&station=126&station=93&station=90&station=25&station=83&station=107&station=156&station=77&station=26&station=70&station=127&station=27&station=132&station=28&station=29&station=30&station=31&station=102&station=32&station=119&station=4&station=80&station=33&station=59&station=105&station=82&station=34&station=72&station=135&station=35&station=76&station=120&station=141&station=109&station=36&station=79&station=71&station=37&station=38&station=39&station=130&station=73&station=40&station=41&station=54&station=69&station=113&station=128&station=42&station=43&station=103&station=116&station=88&station=114&station=3&station=163&station=64&station=115&station=67&station=44&station=133&station=106&station=100&station=121&station=45&station=46&station=61&station=66&station=74&station=60&station=125&station=8&station=47&station=122&station=108&station=5&station=152&station=48&station=68&station=49&station=50&station=91&station=117&station=63&station=150&station=51&station=6&station=52&station=92&station=112&station=131&station=123&station=95&station=53&station=57&station=149&station=148&station=110&variable=ddavt&year=2022&ttype=daily&quick_pick=30_d&begin_date=2022-11-26&end_date=2022-11-26"
ndawn_csv = r"C:\Users\cason\Desktop\GIS5571\Lab3\table.csv"

In [18]:
request = requests.get(url)
data = request.text.splitlines()

In [21]:
df = pd.read_csv(ndawn_csv, skiprows=[0, 1, 2, 4])
print(df)

     Station Name  Latitude  Longitude  ...  Day  Avg Temp  Avg Temp Flag
0             Ada  47.32119  -96.51406  ...   28    51.091            NaN
1             Ada  47.32119  -96.51406  ...   29    45.602            NaN
2             Ada  47.32119  -96.51406  ...   30    40.438            NaN
3             Ada  47.32119  -96.51406  ...   31    49.548            NaN
4             Ada  47.32119  -96.51406  ...    1    49.074            NaN
...           ...       ...        ...  ...  ...       ...            ...
4225      Zeeland  46.01351  -99.68768  ...   22    21.025            NaN
4226      Zeeland  46.01351  -99.68768  ...   23    26.130            NaN
4227      Zeeland  46.01351  -99.68768  ...   24    24.372            NaN
4228      Zeeland  46.01351  -99.68768  ...   25    28.850            NaN
4229      Zeeland  46.01351  -99.68768  ...   26    30.774            NaN

[4230 rows x 9 columns]


In [22]:
header = ['Station Name', 'Latitude', 'Longitude', 'Year', 'Month', 'Day', 'Avg Temp']
df = df[header]
df.head()

,Station Name,Latitude,Longitude,Year,Month,Day,Avg Temp
0,Ada,47.32119,-96.51406,2022,10,28,51.091
1,Ada,47.32119,-96.51406,2022,10,29,45.602
2,Ada,47.32119,-96.51406,2022,10,30,40.438
3,Ada,47.32119,-96.51406,2022,10,31,49.548
4,Ada,47.32119,-96.51406,2022,11,1,49.074


In [23]:
df2 = df.groupby('Station Name').agg(
    AvgtempByStation = pd.NamedAgg(column = 'Avg Temp', aggfunc = pd.Series.mean)
)

df2.to_csv(workspace + r"\AverageTemp.csv")
df2.head()

,AvgtempByStation
Station Name,
Ada,29.798400
Adams,25.555700
Alamo,22.600533
Alvarado,28.123000
Amidon,27.077267


In [24]:
df_merge = df.merge(df2, left_on = 'Station Name', right_on = 'Station Name')
df_merge.head()

,Station Name,Latitude,Longitude,Year,Month,Day,Avg Temp,AvgtempByStation
0,Ada,47.32119,-96.51406,2022,10,28,51.091,29.7984
1,Ada,47.32119,-96.51406,2022,10,29,45.602,29.7984
2,Ada,47.32119,-96.51406,2022,10,30,40.438,29.7984
3,Ada,47.32119,-96.51406,2022,10,31,49.548,29.7984
4,Ada,47.32119,-96.51406,2022,11,1,49.074,29.7984


In [26]:
delete_duplicates = df_merge.drop_duplicates(subset = ['Station Name'])
delete_duplicates.head(100)

,Station Name,Latitude,Longitude,Year,Month,Day,Avg Temp,AvgtempByStation
0,Ada,47.321190,-96.514060,2022,10,28,51.091,29.798400
30,Adams,48.499880,-98.075880,2022,10,28,46.203,25.555700
60,Alamo,48.546520,-103.471860,2022,10,28,41.818,22.600533
90,Alvarado,48.245942,-97.021532,2022,10,28,51.247,28.123000
120,Amidon,46.488440,-103.316290,2022,10,28,49.460,27.077267
...,...,...,...,...,...,...,...,...
2850,Ottertail,46.410950,-95.574300,2022,10,28,49.545,31.475533
2880,Parkers Prairie,46.169410,-95.356020,2022,10,28,50.403,32.110567
2910,Pekin,47.807640,-98.310670,2022,10,28,49.795,27.095667
2940,Perham,46.610490,-95.601880,2022,10,28,49.791,31.076067


In [27]:
delete_duplicates.to_csv(workspace + r"\FinalCSV.csv")

In [28]:
table =  workspace + r"\FinalCSV.csv"

In [29]:
ndawn_points = arcpy.management.XYTableToPoint(table, 
                                               r"C:\Users\cason\Desktop\GIS5571\Lab3\ndawn_stations.shp", 
                                               "Longitude", 
                                               "Latitude", 
                                               None, 
                                               'GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]];-400 -400 1000000000;-100000 10000;-100000 10000;8.98315284119521E-09;0.001;0.001;IsHighPrecision')

In [30]:
arcpy.ga.IDW(ndawn_points, 
             "AvgtempByS", 
             "ndawn_geostat_idw", 
             r"C:\Users\cason\Desktop\GIS5571\Lab3\Lab3\Default.gdb\ndawn_idw", 
             0.0144790399999999, 
             2, 
             "NBRTYPE=Standard S_MAJOR=2.89308477429477 S_MINOR=2.89308477429477 ANGLE=0 NBR_MAX=15 NBR_MIN=10 SECTOR_TYPE=ONE_SECTOR", 
             None)

<Result 'ndawn_geostat_idw'>

In [31]:
arcpy.ga.EmpiricalBayesianKriging(ndawn_points, 
                                  "AvgtempByS", 
                                  "ndawn_geostat_ebkriging", 
                                  r"C:\Users\cason\Desktop\GIS5571\Lab3\Lab3\Default.gdb\ndawn_ebkriging", 
                                  0.0144790399999999, 
                                  "NONE", 
                                  100, 
                                  1, 
                                  100, 
                                  "NBRTYPE=StandardCircular RADIUS=2.89308477429477 ANGLE=0 NBR_MAX=15 NBR_MIN=10 SECTOR_TYPE=ONE_SECTOR", 
                                  "PREDICTION", 
                                  0.5, 
                                  "EXCEED", 
                                  None, 
                                  "POWER")

<Result 'ndawn_geostat_ebkriging'>

In [32]:
ndawn_spline = arcpy.sa.Spline(ndawn_points, 
                               "AvgtempByS", 
                               0.0144790399999999,
                               "REGULARIZED", 
                               0.1, 
                               12); 
ndawn_spline.save(r"C:\Users\cason\Desktop\GIS5571\Lab3\Lab3\Default.gdb\ndawn_spline")